In [ ]:
!pip install optuna

In [3]:
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import joblib

# Read data
data = pd.read_excel('TrainClass.xlsx')
real_data = pd.read_excel('TestClass.xlsx')

# Prepare training data
Y, X = data['FRACASO'], data.drop(['FRACASO', 'CODIGO_EMPRESA'], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Prepare testing data
X_real = real_data.drop(['CODIGO_EMPRESA'], axis=1)
X_real_selected = scaler.transform(X_real)  # Assuming X_real_selected is used for prediction

# Define the objective function for Optuna
def objective(trial):
    # Define hyperparameters to be optimized
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)

    # Number of layers
    num_layers = trial.suggest_int('num_layers', 1, 5)

    # Number of nodes in each layer
    nodes_per_layer = [trial.suggest_int(f'n_nodes_layer_{i}', 8, 128) for i in range(num_layers)]

    # Build the neural network model
    model = Sequential()
    model.add(Dense(units=nodes_per_layer[0], activation='relu', input_dim=40))
    for i in range(1, num_layers):
        model.add(Dense(units=nodes_per_layer[i], activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model with cross-validation
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    cv_scores = []
    for train, val in kfold.split(X_train, Y_train):
        X_train_fold, X_val_fold = X_train[train], X_train[val]
        Y_train_fold, Y_val_fold = Y_train.iloc[train], Y_train.iloc[val]

        model.fit(X_train_fold, Y_train_fold, epochs=10, batch_size=32, validation_data=(X_val_fold, Y_val_fold), verbose=0)

        Y_val_pred = model.predict(X_val_fold)
        auc = roc_auc_score(Y_val_fold, Y_val_pred)
        cv_scores.append(auc)

    # Return the average AUC over folds as the objective value for Optuna
    return np.mean(cv_scores)

# Optimize hyperparameters with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150)

# Get the best hyperparameters
best_params = study.best_params
best_learning_rate = best_params['learning_rate']
best_dropout_rate = best_params['dropout_rate']
best_num_layers = best_params['num_layers']
best_nodes_per_layer = [best_params[f'n_nodes_layer_{i}'] for i in range(best_num_layers)]

# Build the final model with the best hyperparameters
final_model = Sequential()
final_model.add(Dense(units=best_nodes_per_layer[0], activation='relu', input_dim=40))
for i in range(1, best_num_layers):
    final_model.add(Dense(units=best_nodes_per_layer[i], activation='relu'))
final_model.add(Dense(units=1, activation='sigmoid'))
final_model.compile(optimizer=Adam(learning_rate=best_learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

# Train the final model on the entire training set
final_model.fit(X_train, Y_train, epochs=10, batch_size=32, verbose=1)

# Save the trained model to a file
final_model.save('neural_network_model.h5')

# Load model and generate probability CSV
loaded_model = load_model('neural_network_model.h5')
prob = loaded_model.predict(X_real_selected)
prob_df = pd.DataFrame(prob, columns=['Probability'])
prob_df.index = prob_df.index + 1
prob_df.index.name = 'Id'

# Save probability CSV
prob_df.to_csv('probability_predictions.csv')


[I 2023-12-05 20:00:07,735] A new study created in memory with name: no-name-0d53e505-42ce-4d09-8722-98da4f905914


9/9 [==============================] - 0s 2ms/step


[I 2023-12-05 20:00:18,574] Trial 0 finished with value: 0.8812158541941152 and parameters: {'learning_rate': 0.002157822963517337, 'dropout_rate': 0.0856976518730137, 'num_layers': 3, 'n_nodes_layer_0': 23, 'n_nodes_layer_1': 113, 'n_nodes_layer_2': 54}. Best is trial 0 with value: 0.8812158541941152.


9/9 [==============================] - 0s 2ms/step


[I 2023-12-05 20:00:26,743] Trial 1 finished with value: 0.32513680281071583 and parameters: {'learning_rate': 1.6941505256109662e-05, 'dropout_rate': 0.3964314923424408, 'num_layers': 4, 'n_nodes_layer_0': 72, 'n_nodes_layer_1': 66, 'n_nodes_layer_2': 57, 'n_nodes_layer_3': 39}. Best is trial 0 with value: 0.8812158541941152.


...

9/9 [==============================] - 0s 3ms/step


[I 2023-12-05 20:30:10,866] Trial 149 finished with value: 0.8266409749670618 and parameters: {'learning_rate': 0.0010996890713348946, 'dropout_rate': 0.4628987398726083, 'num_layers': 1, 'n_nodes_layer_0': 84}. Best is trial 125 with value: 0.9302356170399648.


Epoch 1/10
45/45 [==============================] - 2s 5ms/step - loss: 0.4458 - accuracy: 0.9033
Epoch 2/10
45/45 [==============================] - 0s 6ms/step - loss: 0.2135 - accuracy: 0.9590
Epoch 3/10
45/45 [==============================] - 0s 6ms/step - loss: 0.1299 - accuracy: 0.9638
Epoch 4/10
45/45 [==============================] - 0s 5ms/step - loss: 0.1132 - accuracy: 0.9687
Epoch 5/10
45/45 [==============================] - 0s 5ms/step - loss: 0.0987 - accuracy: 0.9743
Epoch 6/10
45/45 [==============================] - 0s 6ms/step - loss: 0.0855 - accuracy: 0.9757
Epoch 7/10
45/45 [==============================] - 0s 6ms/step - loss: 0.0747 - accuracy: 0.9784
Epoch 8/10
45/45 [==============================] - 0s 5ms/step - loss: 0.0702 - accuracy: 0.9812
Epoch 9/10
45/45 [==============================] - 0s 5ms/step - loss: 0.0657 - accuracy: 0.9784
Epoch 10/10
45/45 [==============================] - 0s 5ms/step - loss: 0.0621 - accuracy: 0.9777


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


18/18 [==============================] - 0s 2ms/step
